# GPT Actions - Snowflake direct
## はじめに

このページでは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報をよく理解しておいてください：

* [GPT Actionsの概要](https://platform.openai.com/docs/actions)
* [GPT Actions Libraryの概要](https://platform.openai.com/docs/actions/actions-library)
* [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

この特定のGPT Actionは、Snowflake Data Warehouseに接続する方法の概要を提供します。このActionは、ユーザーの質問を受け取り、関連するテーブルをスキャンしてデータスキーマを収集し、その後ユーザーの質問に答えるためのSQLクエリを作成します。

注意：このクックブックは、GPT Actionsのapplication/jsonペイロード制限によって制限されない完全な結果ではなく、[ResultSet SQL statement](https://docs.snowflake.com/en/developer-guide/sql-api/handling-responses#getting-the-data-from-the-results)を返します。本番環境や高度なユースケースでは、CSVファイルを返すためのミドルウェアが必要です。代わりにこのフローを実装するには、[GPT Actions - Snowflake Middleware cookbook](../gpt_actions_library/gpt_action_snowflake_middleware)の手順に従ってください。

### 価値 + ビジネス活用事例

価値: ユーザーは、ChatGPTの自然言語機能を活用して、Snowflakeのデータウェアハウスに直接接続できるようになりました。

使用例:

* データサイエンティストは、テーブルに接続し、ChatGPTのデータ分析機能を使用してデータ分析を実行できます
* 一般的なデータユーザーは、トランザクションデータに対して基本的な質問を行うことができます
* ユーザーは、データや潜在的な異常値についてより多くの可視性を得ることができます

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、アプリケーションの以下のリンクを確認してください：

* アプリケーションウェブサイト: [https://app.snowflake.com/](https://app.snowflake.com/)
* アプリケーションAPI文書: [https://docs.snowflake.com/en/developer-guide/sql-api/intro](https://docs.snowflake.com/en/developer-guide/sql-api/intro)

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：

* Snowflake Data Warehouseをプロビジョニングする
* ChatGPT経由でSnowflakeに認証するユーザーが、必要なロールでデータベース、スキーマ、テーブルにアクセスできることを確認する

## 1. カスタムGPTの設定

### GPT指示の設定

カスタムGPTを作成したら、以下のテキストを指示パネルにコピーしてください。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
**Context**: You are an expert at writing Snowflake SQL queries. A user is going to ask you a question. 

**Instructions**:
1. No matter the user's question, start by running `runQuery` operation using this query: "SELECT column_name, table_name, data_type, comment FROM {database}.INFORMATION_SCHEMA.COLUMNS" 
-- Assume warehouse = "<insert your default warehouse here>", database = "<insert your default database here>", unless the user provides different values 
2. Convert the user's question into a SQL statement that leverages the step above and run the `runQuery` operation on that SQL statement to confirm the query works. Add a limit of 100 rows
3. Now remove the limit of 100 rows and return back the query for the user to see
4. Use the <your_role> role when querying Snowflake
5. Run each step in sequence. Explain what you are doing in a few sentences, run the action, and then explain what you learned. This will help the user understand the reason behind your workflow. 

**Additional Notes**: If the user says "Let's get started", explain that the user can provide a project or dataset, along with a question they want answered. If the user has no ideas, suggest that we have a sample flights dataset they can query - ask if they want you to query that

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。サーバーのURLを、[こちら](https://docs.snowflake.com/en/user-guide/organizations-connect#standard-account-urls)で説明されているように、あなたのSnowflakeアカウント名のURLに`/api/v2`を追加したものに更新してください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
openapi: 3.1.0
info:
  title: Snowflake Statements API
  version: 1.0.0
  description: API for executing statements in Snowflake with specific warehouse and role settings.
servers:
  - url: 'https://<orgname>-<account_name>.snowflakecomputing.com/api/v2'


paths:
  /statements:
    post:
      summary: Execute a SQL statement in Snowflake
      description: This endpoint allows users to execute a SQL statement in Snowflake, specifying the warehouse and roles to use.
      operationId: runQuery
      tags:
        - Statements
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                warehouse:
                  type: string
                  description: The name of the Snowflake warehouse to use for the statement execution.
                role:
                  type: string
                  description: The Snowflake role to assume for the statement execution.
                statement:
                  type: string
                  description: The SQL statement to execute.
              required:
                - warehouse
                - role
                - statement
      responses:
        '200':
          description: Successful execution of the SQL statement.
          content:
            application/json:
              schema:
                type: object
                properties:
                  status:
                    type: string
                  data:
                    type: object
                    additionalProperties: true
        '400':
          description: Bad request, e.g., invalid SQL statement or missing parameters.
        '401':
          description: Authentication error, invalid API credentials.
        '500':
          description: Internal server error.

## 2. Snowflake統合の設定

以下は、このサードパーティアプリケーションでの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

### ChatGPT用のIPホワイトリストの設定
IPによる接続を制限するネットワークポリシーを持つSnowflakeアカウントでは、ChatGPT用の例外を追加する必要がある場合があります。
* [ネットワークポリシー](https://docs.snowflake.com/en/user-guide/network-policies)に関するSnowflakeドキュメントを確認してください
* Snowflake Worksheetsにアクセスしてください
* [こちら](https://platform.openai.com/docs/actions/production/ip-egress-ranges#ip-egress-ranges)に記載されているChatGPT IPエグレス範囲を使用してネットワークルールを作成してください
* 対応するネットワークポリシーを作成してください

In [ ]:
## ChatGPT IP ranges available at https://openai.com/chatgpt-actions.json
CREATE NETWORK RULE chatgpt_network_rule
  MODE = INGRESS
  TYPE = IPV4
  VALUE_LIST = ('23.102.140.112/28',...,'40.84.180.128/28');

CREATE NETWORK POLICY chatgpt_network_policy
  ALLOWED_NETWORK_RULE_LIST = ('chatgpt_network_rule');

ネットワークポリシーは、アカウント、セキュリティ統合、およびユーザーレベルで適用できます。最も具体的なネットワークポリシーが、より一般的なネットワークポリシーを上書きします。これらのポリシーの適用方法によっては、セキュリティ統合に加えて個々のユーザーのポリシーを変更する必要がある場合があります。この問題に直面した場合、Snowflakeのエラーコード390422または一般的な「Invalid Client」エラーが発生する可能性があります。

### セキュリティ統合の作成
* Snowflake OAuth概要を確認してください：[https://docs.snowflake.com/en/user-guide/oauth-snowflake-overview](https://docs.snowflake.com/en/user-guide/oauth-snowflake-overview)
* [Security Integration](https://docs.snowflake.com/en/sql-reference/sql/create-security-integration-oauth-snowflake)を通じて新しいOAuth認証情報を作成してください - SnowflakeのリダイレクトURIはセキュリティ統合と1対1でマッピングされるため、各OAuthアプリ/カスタムGPTごとに新しいものが必要になります

In [ ]:
CREATE SECURITY INTEGRATION CHATGPT_INTEGRATION
  TYPE = OAUTH
  ENABLED = TRUE
  OAUTH_CLIENT = CUSTOM
  OAUTH_CLIENT_TYPE = 'CONFIDENTIAL'
  OAUTH_REDIRECT_URI = 'https://oauth.pstmn.io/v1/callback' --- // this is a temporary value while testing your integration. You will replace this with the value your GPT provides
  OAUTH_ISSUE_REFRESH_TOKENS = TRUE
  OAUTH_REFRESH_TOKEN_VALIDITY = 7776000
  NETWORK_POLICY = chatgpt_network_policy; --- // this line should only be included if you followed step 1 above

<details>
  <summary>オプション: ネットワークルール設定の自動化</summary>
  
  現在、ChatGPTで使用されるエグレスIPアドレスは100個を超えています。このリストは不定期に、かつ事前告知なしに更新されます。最新の状態を保つために、毎日リストを取得してネットワークルールに適用することができます。

  ### OpenAIへのアウトバウンドトラフィックを許可するネットワークルール
  ```sql
  CREATE OR REPLACE NETWORK RULE chatgpt_actions_rule
  MODE = EGRESS       -- outbound
  TYPE = HOST_PORT
  VALUE_LIST = ('openai.com:443');
  ```
   ### ルールを適用するアクセス統合
  ```sql
  CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION chatgpt_actions_integration
  ALLOWED_NETWORK_RULES = (chatgpt_actions_rule)
  ENABLED = TRUE;
  ```

  ### IP範囲を取得するUDF
  ```sql
  CREATE OR REPLACE FUNCTION getChatGPTActionsAddresses()
  RETURNS ARRAY  -- array<varchar>
  LANGUAGE PYTHON
  RUNTIME_VERSION = 3.10
  PACKAGES = ('requests')
  EXTERNAL_ACCESS_INTEGRATIONS = (chatgpt_actions_integration)
  HANDLER = 'get_ip_address_ranges'
AS
$$
import requests

def get_ip_address_ranges():
    resp = requests.get("https://openai.com/chatgpt-actions.json", timeout=10)
    resp.raise_for_status()
    data = [entry["ipv4Prefix"] for entry in resp.json().get("prefixes", []) if "ipv4Prefix" in entry]
    return data
$$;
  ```
  ### ネットワークルールを更新するプロシージャ
  ```sql
  CREATE OR REPLACE PROCEDURE update_chatgpt_network_rule()
  RETURNS STRING
  LANGUAGE SQL
AS
$$
DECLARE
  ip_list STRING;
BEGIN
  -- VALUE_LISTで使用するためにIPを適切にクォートする
  ip_list := '''' || ARRAY_TO_STRING(getChatGPTActionsAddresses(), ''',''') || '''';

  -- ルールを更新する動的SQLを実行
  EXECUTE IMMEDIATE
    'ALTER NETWORK RULE chatgpt_network_rule SET VALUE_LIST = (' || ip_list || ')';

  RETURN 'chatgpt_network_rule updated with ' || ARRAY_SIZE(getChatGPTActionsAddresses()) || ' entries';
END;
$$;
  ```

  ### プロシージャの呼び出し
  ```sql
  CALL update_chatgpt_network_rule();
  ```

  ### 太平洋時間の毎日午前6時にプロシージャを実行
  ```sql
  CREATE OR REPLACE TASK auto_update_chatgpt_network_rule
  WAREHOUSE = COMPUTE_WH
  SCHEDULE = 'USING CRON 0 6 * * * America/Los_Angeles'
AS
  CALL update_chatgpt_network_rule();
  ```
</details>

## 3. GPT Action認証の設定
### Snowflakeから重要な情報を収集する
* OAuth Client ID、Auth URL、Token URLを取得する

In [ ]:
DESCRIBE SECURITY INTEGRATION CHATGPT_INTEGRATION;

必要な情報は以下の3行にあります：

![../../../images/snowflake_direct_oauth.png](../../../images/snowflake_direct_oauth.png)

* `SHOW_OAUTH_CLIENT_SECRETS`を使用してOAuthクライアントシークレットを取得する

In [ ]:
SELECT 
trim(parse_json(SYSTEM$SHOW_OAUTH_CLIENT_SECRETS('CHATGPT_INTEGRATION')):OAUTH_CLIENT_ID) AS OAUTH_CLIENT_ID
, trim(parse_json(SYSTEM$SHOW_OAUTH_CLIENT_SECRETS('CHATGPT_INTEGRATION')):OAUTH_CLIENT_SECRET) AS OAUTH_CLIENT_SECRET;

今は[PostmanでSnowflake統合をテスト](https://community.snowflake.com/s/article/How-to-configure-postman-for-testing-SQL-API-with-OAuth)するのに良いタイミングです。セキュリティ統合でネットワークポリシーを設定した場合は、テストに使用しているマシンのIPが含まれていることを確認してください。

### GPT Action認証でOAuth値を設定する

ChatGPTで「Authentication」をクリックし、「OAuth」を選択します。以下の情報を入力してください。

| フォームフィールド | 値  |
| -------- | -------- |
| Authentication Type   | OAuth   |
| Client ID   | SHOW_OAUTH_CLIENT_SECRETSからのOAUTH_CLIENT_ID   |
| Client Secret   | SHOW_OAUTH_CLIENT_SECRETSからのOAUTH_CLIENT_SECRET   |
| Authorization URL   | DESCRIBE SECURITY INTEGRATIONからのOAUTH_AUTHORIZATION_ENDPOINT |
| Token URL   | DESCRIBE SECURITY INTEGRATIONからのOAUTH_TOKEN_ENDPOINT   |
| Scope   | session:role:CHATGPT_INTEGRATION_ROLE*   |
| Token Exchange Method   | Default (POST Request)   |

*Snowflakeのスコープは`session:role:<your_role>`の形式でロールを渡します（例：`session:role:CHATGPT_INTEGRATION_ROLE`）。このフィールドを空のままにしてGPTの指示でロールを指定することもできますが、ここに追加することでOAuth同意リクエストに含まれるため、より信頼性が高くなる場合があります。

### 4. Snowflake統合のリダイレクトURIを更新する

ChatGPTで認証を設定したら、アプリケーションで以下の手順に従ってActionを完了してください。

* GPT ActionからコールバックURLをコピーする
* セキュリティ統合のリダイレクトURIを、ChatGPTで提供されたコールバックURLに更新する

In [ ]:
ALTER SECURITY INTEGRATION CHATGPT_INTEGRATION SET OAUTH_REDIRECT_URI='https://chat.openai.com/aip/<callback_id>/oauth/callback';

### FAQ & トラブルシューティング

* このガイドは一般的な概念を説明することを目的としており、参考目的でのみ提供されています。サードパーティAPI統合に関する完全なサポートを提供することはできません。
* YAMLを更新するとコールバックURLが変更される可能性があります。変更を行う際は、URLが正しいことを再確認してください。
* _コールバックURLエラー:_ ChatGPTでコールバックURLエラーが発生した場合は、上記のPost-Action Stepsに細心の注意を払ってください。アクションが正しく認証されるように、コールバックURLをSecurity Integrationに直接追加する必要があります。
* _スキーマが間違ったウェアハウスやデータベースを呼び出す:_ ChatGPTが間違ったウェアハウスやデータベースを呼び出す場合は、以下のいずれかをより明確にするように指示を更新することを検討してください：(a) どのウェアハウス/データベースを呼び出すべきか、または (b) クエリを実行する前にユーザーがそれらの正確な詳細を提供することを要求する

_私たちに優先的に取り組んでほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。_